過去event+アソシエーションルール+人気

In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm
import random
random.seed(64)

In [2]:
path_input = {}
path_output_event = {}
path_output_popul = {}
path_output_assoc = {}

for i in ["train1", "valid"]:
    path_input[i] = f"/kaggle/input/otto-make-data-train-test-split/{i}.jsonl"
    path_output_event[i] = f"past_event_{i}.pickle"
    path_output_popul[i] = f"population_{i}.pickle"
    path_output_assoc[i] = f"association_{i}.pickle"
    
path_input_assoc = {}
for i in ["clicks", "carts", "orders"]:
    path_input_assoc[i]= f"/kaggle/input/otto-cf-association-{i}/association_rules_{i}.pickle"
    
sample_size = 150000
num_top_item = 10 # 上位いくつまでのitemを候補とするか
debug = False

n_session = {"train1": 2237925, "valid": 1735763}

In [3]:
# アソシエーションルール
assoc = {}
for i in ["clicks", "carts", "orders"]:
    assoc[i] = pd.read_pickle(path_input_assoc[i]).values
assoc["orders"]

array([[frozenset({756588}), frozenset({231487}), 8.00701517845958e-05,
        0.23396226415094343, 469.3313324860691],
       [frozenset({231487}), frozenset({756588}), 8.00701517845958e-05,
        0.16062176165803108, 469.331332486069],
       [frozenset({231487}), frozenset({1111967}),
        1.0331632488334942e-05, 0.02072538860103627, 98.45462347817795],
       ...,
       [frozenset({150889}), frozenset({202224, 933686, 1446430}),
        2.3246173098753617e-05, 0.30508474576271183, 7874.450847457627],
       [frozenset({202224}), frozenset({150889, 933686, 1446430}),
        2.3246173098753617e-05, 0.3157894736842105, 8150.747368421053],
       [frozenset({1446430}), frozenset({202224, 150889, 933686}),
        2.3246173098753617e-05, 0.29508196721311475, 11424.408196721311]],
      dtype=object)

In [4]:
# event実績のあるaid+アソシエーションルールに該当するaid
for i in ["train1", "valid"]:
    dict_count = {"clicks": {}, "carts": {}, "orders": {}}
    dict_past_event = {"clicks": {}, "carts": {}, "orders": {}}
    dict_assoc = {"clicks": {}, "carts": {}, "orders": {}}
    chunks = pd.read_json(path_input[i], lines = True, chunksize = sample_size)
    for df_chunk in tqdm(chunks, total = n_session[i]//sample_size+1):
        for j, session in df_chunk.iterrows():
            session = session.to_dict()
            list_event = session["events"][:session["cutoff"]] # cutoff以前のeventのみ使える
            # 各aidの発生回数をカウント
            for event in list_event:
                dict_count[event["type"]][event["aid"]] = dict_count[event["type"]].get(event["aid"], 0) + 1
            # セッションごとの実績aid
            for types in ["clicks", "carts", "orders"]:
                dict_past_event[types][session["session"]] = list(set([event["aid"] for event in list_event if event["type"] == types]))
            list_aid = list(set([event["aid"] for event in list_event]))
            set_aid = set(list_aid)
            # event実績のあるaid+アソシエーションルール
            for types in ["clicks", "carts", "orders"]:
                # アソシエーションルール
                list_assoc = assoc[types][assoc[types][:,0] <= set_aid, 1]
                dict_assoc[types][session["session"]] = list(set(sum([list(x) for x in list_assoc], [])))
            if debug: break
        if debug: break
    if debug: break
    # event_typeごとに回数の多い順にソートしてkeyだけ取り出す
    dict_popul = {}
    for types in ["clicks", "carts", "orders"]:
        dict_popul[types] = [j[0] for j in sorted(dict_count[types].items(), key = lambda x : x[1], reverse = True)]
        dict_popul[types] = dict_popul[types][:num_top_item]
    pd.to_pickle(dict_popul, path_output_popul[i])
    pd.to_pickle(dict_past_event, path_output_event[i])
    pd.to_pickle(dict_assoc, path_output_assoc[i])    

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
def process_session():
            list_event = session["events"][:session["cutoff"]] # cutoff以前のeventのみ使える
            # 各aidの発生回数をカウント
            for event in list_event:
                dict_count[event["type"]][event["aid"]] = dict_count[event["type"]].get(event["aid"], 0) + 1
            # セッションごとの実績aid
            list_aid = list(set([event["aid"] for event in list_event]))
            dict_past_event[session["session"]] = list_aid
            set_aid = set(list_aid)
            # event実績のあるaid+アソシエーションルール
            for types in ["clicks", "carts", "orders"]:
                # アソシエーションルール
                list_assoc = arr[arr[:,0] <= set_aid, 1]
                dict_assoc[types][session["session"]] = list(set(sum([list(x) for x in list_assoc], [])))


In [6]:
if debug:
    import line_profiler
    pr = line_profiler.LineProfiler()#LineProfilerクラスのインスタンス生成
    pr.add_function(process_session)#対象の関数をadd_functionで登録
    pr.runcall(process_session)#runcallで実行
    pr.print_stats()#結果を表示